# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py", base_dir=project_root)
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 10:49:06,305 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [4]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
You are a senior QA engineer specializing in FastAPI API test authoring. You will generate **only** Python pytest code for happy path tests of the provided application.

CONTEXT (FastAPI app code):
```
{app_code}
```

TARGET ENDPOINTS TO TEST:
1. POST /users/  -> Should create a new user. Expect HTTP 201, response JSON includes: user_id (int), full_name (str), email (str), role (str), hire_date (ISO 8601 or date), created_at, updated_at.
2. GET /users/   -> Should return a list (possibly empty or containing previously created users). Expect HTTP 200, list of user objects with the same shape.

REQUIREMENTS:
- Use pytest.
- Use from fastapi.testclient import TestClient
- Instantiate TestClient against the FastAPI app imported from app.main (e.g. `from app.main import app`).
- Each test must be independent; do NOT rely on ordering. If needed, create a user within the same test before asserting GET /users/.
- Use unique emails per test (e.g., f"user_{{uuid.uuid4().hex[:8]}}@example.com") to avoid clashes.
- Assert specific status codes.
- Assert essential response fields and types.
- Do NOT include any extraneous comments beyond what is helpful.
- Do NOT print anything.
- No backticks, no markdown; return pure Python code only.
- Avoid sleeping, network calls, or accessing external files.
- Keep tests concise but clear; 1 test for POST success, 1 test for GET listing.
- If the POST response role field is string, assert it's one of expected role values or non-empty (based on provided code; adapt accordingly).
- If the application sets timestamps, you can assert presence (key in response) rather than exact value.

OUTPUT FORMAT:
Return a single Python file content containing:
- All imports at top.
- Two test functions: test_create_user_happy_path, test_get_users_happy_path.

Do NOT include anything else but the Python code.
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py", base_dir=project_root)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
import datetime
import uuid

from fastapi.testclient import TestClient

# Assuming the FastAPI app instance is in `app.main`
# This might need to be adjusted based on the actual project structure.
from app.main import app

client = TestClient(app)


def test_create_user_happy_path():
    """
    Tests the successful creation of a new user via the POST /users/ endpoint.
    """
    unique_email = f"testuser_{uuid.uuid4().hex[:8]}@example.com"
    user_data = {
        "full_name": "Jane Doe",
        "email": unique_email,
        "sso_identifier": f"sso_{uuid.uuid4().hex[:12]}",
        "role": "employee",
        "manager_id": None,
        "hire_date": datetime.date.today().isoformat()
    }

    response = client.post("/users/", json=user_data)

    assert response.status_code == 201

    data = response.json()

    # Assert shape and types of the response
    assert "user_id" in data
    assert isinstance(data["user_id"], int)
    assert "full_na

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [7]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
You are a senior QA engineer specializing in FastAPI negative & edge case testing. Generate ONLY Python pytest code (no markdown, no commentary) that adds edge case tests for the provided application.

CONTEXT (FastAPI app code):
```
{app_code}
```

EDGE CASES TO TEST (exactly two test functions):
1. Duplicate Email on User Creation:
   - Flow:
     * Create a user via POST /users/ (expect 201).
     * Re-submit the exact same payload (same email) via POST /users/ (expect 400).
   - Assert:
     * First response status == 201.
     * Second response status == 400.
     * Second response JSON contains {{"detail": "An account with this email already exists."}} (exact match).
2. Get Non-Existent User:
   - Flow:
     * Create a user (to confirm normal behavior).
     * Compute a guaranteed-missing ID = created_user_id + 99999.
     * GET /users/{{missing_id}} (expect 404).
   - Assert:
     * Response status == 404.
     * JSON["detail"] == f"User with ID {{missing_id}} not found".

REQUIREMENTS:
- Use: from fastapi.testclient import TestClient
- Import: uuid, pytest
- Import the FastAPI app as: from app.main import app
- Do NOT rely on test execution order; each test self-contained.
- Generate unique user emails with uuid (e.g., f"user_{{uuid.uuid4().hex[:8]}}@example.com").
- Provide exactly two test functions named:
    * test_create_user_duplicate_email_rejected
    * test_get_user_not_found_returns_404
- For the initial successful creation in each test, minimally assert essential fields: user_id (int), email (str), full_name (str), role (str).
- No prints, no sleeps, no external file access, no extraneous comments.
- Pure Python output. No backticks in final output.

OUTPUT FORMAT:
Return a single Python file content containing:
- All necessary imports at top.
- The two test functions only.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
    save_artifact(cleaned_edge_case_tests, "tests/test_main_edge_cases.py", base_dir=project_root)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
import pytest
import uuid
import datetime
from fastapi.testclient import TestClient

from app.main import app

client = TestClient(app)


def test_create_user_duplicate_email_rejected():
    unique_email = f"user_{uuid.uuid4().hex[:8]}@example.com"
    user_payload = {
        "full_name": "Duplicate User",
        "email": unique_email,
        "sso_identifier": uuid.uuid4().hex,
        "role": "employee",
        "manager_id": None,
        "hire_date": datetime.date.today().isoformat(),
    }

    # First request: Create the user successfully
    response1 = client.post("/users/", json=user_payload)
    assert response1.status_code == 201
    data1 = response1.json()
    assert isinstance(data1["user_id"], int)
    assert data1["email"] == unique_email
    assert data1["full_name"] == "Duplicate User"
    assert data1["role"] == "employee"

    # Second request: Attempt to create the same user again
    response2 = client.post("/users/", json=user

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [9]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
# Provides: in-memory SQLite engine, SessionLocal, override get_db, client & db_session fixtures.
db_fixture_prompt = f"""
You are a senior Python QA engineer. Generate ONLY Python code (no markdown) for a pytest fixture module enabling isolated, in-memory database tests for the provided FastAPI app.

CONTEXT (FastAPI app code):
```
{app_code}
```

GOAL:
Create a file tests/conftest.py containing:
1. Imports (pytest, uuid, datetime, sqlalchemy components, fastapi.testclient.TestClient).
2. Create an in-memory SQLite SQLAlchemy engine: create_engine("sqlite:///:memory:", connect_args={{"check_same_thread": False}}).
3. Import Base metadata & get_db dependency pieces from the app (derive from app.database / app.main code). If Base not directly exposed, reconstruct via importing the models or using declarative base from existing code.
4. Create a SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine).
5. A fixture `db_session` (scope="function") that: creates all tables (Base.metadata.create_all(engine)), yields a SQLAlchemy session, then rolls back any pending transaction and drops all tables after test for full isolation (Base.metadata.drop_all(engine)). Ensure the session is closed in a finally block.
6. Override the FastAPI dependency get_db to use sessions from SessionLocal inside the test context. Implement a `override_get_db` generator that yields db and closes it.
7. Apply app.dependency_overrides[get_db] = override_get_db BEFORE yielding the TestClient fixture.
8. Fixture `client` (scope="function") returning TestClient(app) using the overridden dependency; ensures cleanup of dependency_overrides after test.
9. Optional helper: `create_user(factory)` function for tests to re-use, but keep minimal.

REQUIREMENTS:
- Two fixtures: db_session, client.
- Use typing Optional where helpful.
- Avoid any external file I/O.
- No prints, no sleeps.
- Pure Python code. No backticks in the final output.
- Ensure dropping tables does not raise if already gone (normal call sequence).
- Keep comments concise and purposeful.

OUTPUT FORMAT:
Return ONLY the Python code for tests/conftest.py.
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
You are a senior QA engineer. Refactor happy path tests for the provided FastAPI app to use the fixtures from tests/conftest.py (client fixture, and implicit db isolation) rather than instantiating TestClient directly.

CONTEXT (FastAPI app code):
```
{app_code}
```

EXISTING HAPPY PATH TESTS (logic reference ONLY, do NOT copy imports):
```
{cleaned_tests}
```

GOAL:
Produce a new file tests/test_main_with_fixture.py containing:
- Imports: uuid, pytest, datetime (if needed for hire_date), typing (Optional), and rely on the client fixture.
- No direct TestClient(app) instantiation; use the injected `client` fixture parameter.
- Two test functions: test_create_user_happy_path_isolated, test_get_users_happy_path_isolated.

TEST REQUIREMENTS:
1. test_create_user_happy_path_isolated:
   - Generate unique email using uuid.
   - POST /users/ payload with valid role (choose from new_hire, manager, hr_admin).
   - Assert 201 status.
   - Assert required fields exist and basic type checks (user_id int, email str, role str, full_name str, hire_date string/date, created_at, updated_at present).
2. test_get_users_happy_path_isolated:
   - Optionally create one user first (to ensure non-empty list) using same POST logic (unique email).
   - GET /users/ -> status 200.
   - Assert response is list; if user created, confirm at least one element with expected keys.

GENERAL REQUIREMENTS:
- Use edge marker pattern if desired? (Skip; keep simple.)
- No prints, sleeps, external I/O.
- Pure Python only (no markdown, no backticks).
- Independent tests (each gets fresh DB due to fixtures).
- Keep comments minimal.

OUTPUT FORMAT:
Return ONLY Python code for tests/test_main_with_fixture.py.
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", base_dir=project_root)
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
import pytest
from typing import Generator

from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session

# Import the main FastAPI app, the dependency, and the SQLAlchemy Base
# This assumes a project structure where 'app' is a package at the root.
from app.main import app
from app.database import get_db, Base

# --- TEST DATABASE SETUP ---

# Define the in-memory SQLite database URL for testing
SQLALCHEMY_DATABASE_URL = "sqlite:///:memory:"

# Create a SQLAlchemy engine for the test database
# connect_args is needed for SQLite to allow multi-threaded access
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

# Create a sessionmaker to generate new Session objects for the test database
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


# --- PYTEST FIXTURES ---

@pytest.fixture(scope="function")
def db_s

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.